# **Federated learning server without local data sharing**

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd

import json

import sys
sys.path.append("../")
import neural_network
import utils

import infer_tools
import participant

import concurrent.futures
#from multiprocessing.pool import ThreadPool
#from multiprocessing.pool import Pool
import subprocess
import requests

from tqdm.notebook import trange

from datetime import datetime
import time

tf.config.set_visible_devices([], 'GPU')

2023-02-02 22:22:21.771024: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-02 22:22:21.776808: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-02 22:22:21.777034: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)

## **Preparations**

In [2]:
with open("../../02_data/filtered_vehicles.json", "r") as f:
    filtered_vehicles = json.load(f)
with open("../../dockeroutput/day_0/vehicle_maps.json", "r") as f:
    vehicle_maps = json.load(f)
    veh_to_idx_map = vehicle_maps["vehicle_to_idx_map"]
    idx_to_veh_map = vehicle_maps["idx_to_vehicle_map"]

In [3]:
filtered_vehicles = filtered_vehicles["vehicles"]

In [4]:
settings = {
    "min": 18220,
    "max": 46800,
    "mean": 0.59,
    "std": 0.35302
}

PARKING_ID_LIST = [i for i in range(1059, 1186+1)]

In [5]:
test_t = np.arange(14460, 50400+1, 60)
test_t = utils.normalize(test_t, settings["min"], settings["max"])

In [6]:
oracle = pd.read_csv("../../02_data/oracle.csv")

In [7]:
plist=[i for i in range(1059,1186+1)]
plist.remove(1148)
settings["parkings"] = plist
parking_testset = None
for parking in plist:
    id_encoding = utils.one_hot_encoder([parking]*len(test_t), PARKING_ID_LIST)
    test_d = np.hstack([id_encoding, test_t.reshape(len(test_t), 1)])
    if parking_testset is None:
        parking_testset = test_d
    else:
        parking_testset = np.vstack([parking_testset, test_d])

In [8]:
with open("../../02_data/known_parkings.json", "r") as f:
    known_parkings_by_vehicles = json.load(f)
with open("../../02_data/known_moving_times.json", "r") as f:
    known_moving_times = json.load(f)

## **Learning**

In [9]:
NUM_VEHICLES = 50
NUM_WORKERS = 2

for i in range(NUM_WORKERS):
    subprocess.Popen(["flask", "--app", "flask_participant.py", "run",
                      "-p", "%d"%(5000+i)], stdout=subprocess.DEVNULL);

In [10]:
def train_participant(parameters):
    def _train(participant, model_json, veh_num, day, parking_test_set):
        port_number = 5000+participant
        req = {
            "model": model_json,
            "vehicle": veh_num,
            "day": day,
            "test_data": parking_test_set.tolist()
        }
        print("http://localhost:%d/train"%port_number)
        result = requests.post("http://localhost:%d/train"%port_number, json.dumps(req)).json()
        #result = json.load(result)
        model = keras.models.model_from_json(result["model"])
        model.compile()
        weights = model.get_weights()
        num_samples = result["num_samples"]
        occups = result["eval_results"]
        return weights, num_samples, occups

    def _eval(veh_id, occup, baseline_occup, oracle):
        true_parkings = known_parkings_by_vehicles[veh_id]
        true_time = known_moving_times[veh_id]
        return infer_tools.evaluate_performance(occup, oracle, baseline_occup,
                                                settings, true_parkings, true_time)
    
    time.sleep(np.random.uniform(0, 3, 1)[0])
    partip = parameters[0]
    fedmodel = parameters[1]
    veh_num = veh_to_idx_map[parameters[2]]
    day = parameters[3]
    #print("train %s %d th day"%(parameters[2], day))
    baseline = parameters[4]
    oracle = parameters[5]
    parking_test_set = parameters[6]
    weights, samp_num, occup = _train(partip, fedmodel.model.to_json(), veh_num, day, parking_test_set)
    #print("eval")
    results = _eval(parameters[2], occup, baseline, oracle)
    
    return weights, samp_num, results[0], results[1]

In [11]:
def fed_avg(parameters, weights):
        new_params = []
        for layer_i in range(len(parameters[0])):
            
            new_params.append(
                np.average([parameters[i][layer_i] for i in range(len(parameters))],
                           weights=weights,
                           axis=0
                )
            )
        return new_params

In [12]:
fed_model = neural_network.NeuralNetwork()

fed_weights = fed_model.model.get_weights()

#pool = concurrent.futures.ThreadPoolExecutor(max_workers=16)
#pool = ThreadPool(16)
#pool = Pool(16)

position_accuracies = []
time_accuracies = []


for day in trange(55):
    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("DAY%d\tcurrent Time ="%day, current_time)
    selected_participants = np.random.choice(np.arange(0, len(filtered_vehicles)), NUM_VEHICLES, replace=False)
    selected_vehicles = [filtered_vehicles[i] for i in selected_participants]
    
    params = []
    baseline_occups = fed_model.model.predict(parking_testset).reshape(len(parking_testset))
    for i in range(len(selected_vehicles)):
        #a car may not have information from any of the parking lots
        if selected_vehicles[i] in known_parkings_by_vehicles:
            params.append([i%NUM_WORKERS, fed_model, selected_vehicles[i], day, baseline_occups,
                       oracle, parking_testset])
    
    #print(params)
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as pool:
        results = pool.map(train_participant, params)#pool.imap_unordered(train_participant, params)
        weights = []
        num_samps = []
        p_acc, t_acc = [], []
        for r in results:
            weights.append(r[0])
            num_samps.append(r[1])
            p_acc.append(r[2])
            t_acc.append(r[3])
        position_accuracies.append(p_acc)
        time_accuracies.append(t_acc)
    
    fed_weights = fed_avg(weights, num_samps)
    fed_model.model.set_weights(fed_weights)

2023-02-02 22:22:25.019214: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


  0%|          | 0/55 [00:00<?, ?it/s]

DAY0	current Time = 22:22:25
2382/2382 [==============================] - 1s 386us/step
http://localhost:5000/train
http://localhost:5001/train
http://localhost:5000/train
http://localhost:5001/train
http://localhost:5000/train
http://localhost:5001/train
http://localhost:5000/train
http://localhost:5001/train
http://localhost:5000/train
http://localhost:5001/train
http://localhost:5000/train
http://localhost:5001/train
http://localhost:5000/train
http://localhost:5001/train
http://localhost:5000/train
http://localhost:5001/train
http://localhost:5000/train
http://localhost:5001/train
http://localhost:5000/train
http://localhost:5001/train
http://localhost:5000/train
http://localhost:5001/train
http://localhost:5000/train
http://localhost:5001/train
http://localhost:5000/train
http://localhost:5001/train
http://localhost:5000/train
http://localhost:5001/train
http://localhost:5000/train
http://localhost:5001/train
http://localhost:5000/train
http://localhost:5001/train
http://localhost

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
save = {
    "position_accuracies": position_accuracies,
    "time_accuracies": time_accuracies
}

In [ ]:
with open("../../02_data/fed_accuracies2.json", "w") as f:
    json.dump(save, f)

In [ ]:
fed_model.model.save("../../04_nn_models/fed_noshare2")